## FP data types

`fp-ts` provides us with several data types that encapsulate different "behaviours" we might need when handling __values__ in our app.   
We can say that we are enhancing our values with different capabilities. Let's take a look at what those are:

Note that the actual implementation in `fp-ts` might be slightly different than presented here, as we're aiming for a simple short summary



### I like my `Option`

This is the simplest data type and it provides a representation of an *optional value*   
We define it as:


In [3]:
type None = { _tag: 'None' }
type Some<A> = {
    _tag: 'Some'
    value: A
}

type Option<A> = None | Some<A>

> **NOTE** 
> 
> We use a `_tag` field to create a tagged union so we can narrow the types when needed.  
> This is a common pattern we'll use for most data types.   


In TypeScript, we can usually defined a nullable or optional value with `A | undefined`.  
`Option` provides us with a different way to achieve the same functionality without ever having to deal with `undefined`.   
There are some interesting implications to this, for example:


In [4]:
type ObjUndefined = { one: number, two?: number }
type ObjOptional = { one: number, two: Option<number>}

In the first one, `two` not existing might have a different meaning than it being merely `undefined`, which can sometimes cause bugs.  
On the other hand, `Option` will always be defined, but it might represent the absence of a value in the `None` case. So if we wanted to distinguish between "not existing" and "being undefined" we'd have to model that in a more explicit way, which should help prevent any bugs.   

Despite this, most TS code is prolific with nullable values using `undefined`, so we're probably better off sticking with the native solution. Which is why we have a custom data type `Nullable` for better interop with normal TS code.  
We define it as:


In [5]:
type Nullable<A> = A | undefined

Nevertheless, `fp-ts`  will always use `Option`, so we either need to convert it to a `Nullable`, or just continue using it.   


A fairly common example is wanting to modify a value only if it's not `undefined`, something like:

In [6]:
let value: string | undefined = "foo"

const exclam = (str: string) => str + "!"
value  = value !== undefined ? exclam(value) : undefined 

foo!


We can extract that `undefined` check into a function we'll call `map` 

In [7]:
const map = <A, B>(ma: Nullable<A>, f: (a:A) => B) => ma !== undefined ? f(ma) : undefined


> **Note**  
>  
> 
> When we typically want to denote a variable for a value of any simple type `A`, we call it `a` (or `b`, `c`, `d` and so on).   
> Similarly, when referring to a function, we use `f`, `g`, `h` and so on.  
> More interestingly, when we refer to one of these "enhanced" values like `Nullable<A>`, we use `ma`, `mb`, `mc` and so on.   
> We'll cover why `m<x>` notation later, but it should at least be clear that `m` refers to the "container" and `a`, `b` etc, refer to the "inner" value   
>    

We can now refactor the above to:

In [8]:
let value: Nullable<string> = "foo"

const exclam = (str: string) => str + "!"
value = map(value, exclam)

foo!


> **Tip**  
>
> just using `map` can be ambiguous. We would typically import it, so it's good practice to use a single letter to group together the fns from the same module:
>


In [9]:
import * as N from './lib/nullable'
import * as O from 'fp-ts/Option'

value = N.map(value, exclam)

foo!!


For completion, let's add the implementation of `map` for `Option`

In [10]:
const optMap = <A, B>(ma: O.Option<A>, f: (a: A) => B): O.Option<B> => ma._tag === "None" ? ma: { _tag: "Some", value: f(ma.value) }

// or, using some utilities from `fp-ts`
const optMap_ = <A, B>(ma: O.Option<A>, f: (a: A) => B): O.Option<B> => O.isNone(ma) ? ma: O.of(f(ma.value))

const optVal: O.Option<number> = O.some(1)
const noVal: O.Option<number> = O.none

const increment = x => x + 1

console.log(optMap(optVal, increment))
console.log(optMap(noVal, increment))


{ _tag: 'Some', value: 2 }
{ _tag: 'None' }


## Love me some curry

If you've checked the types for `O.map`, you might have noticed that it's slightly different than the implementation above. 

In [ ]:
type optionMap = <A, B>(f: (a: A) => B) => (ma: O.Option<A>) => O.Option<B>

This is called __currying__, when insead of having multiple parameters to a function, we return another function that then takes the second parameter as its argument.   
Notice that we also flipped the order of the arguments. Now the firts thing we provide to `map` is the mapping function, and only then the data we want to map. These functions are called curried, data last functions.   
They are a bit more flexible and expressive as they allows us to eschew "intermediate" variables.  
As an example: 

In [ ]:
import * as F from 'fp-ts/function'

const hello = O.some("hello")

//utilities
const toUpper = str => str.toUpperCase()
const repeat = str => str + " && " + str

const uppercased = optMap(hello,toUpper)
const exclamUppercased = optMap(uppercased, exclam) 
const repeatedExclamUppercased = optMap(exclamUppercased, repeat)

console.log(repeatedExclamUppercased)


const pipedVersion = F.pipe(
    hello,
    O.map(toUpper),
    O.map(exclam),
    O.map(repeat)
)

console.log(pipedVersion)

Obviously this is exaggerated, but the idea is that we let the functions tell us what's happening to the value, rather than defining a bunch of variables that are not used for anything other than being passed to the next "step" of the pipeline

## I `Promise` this won't hurt

Typescript itself already provides one of these "enhancing" data types: `Promise`!

Promises are used for async behaviour, in this context we can think of them as value "enhancers" providing the capability of asynchronous computations. And if fact, they even provide a `map` method: `.then()`!

Of course `.then` is a bit more involved, but it does achieve the same concept for `Promise`s as `map` in `Option`s

In [ ]:
const stringPromise: Promise<string> = Promise.resolve("hello")

const exclamPromise = stringPromise.then(exclam)

console.log(await exclamPromise)

Notice that `.then` is a method, we're more interested in curried, data last functions so that we have an uniform API between `Option` and `Promise` and other data types we'll learn about later   
The `lib/promise` module provides wrapper implementations for this. We also separate `.then` into `map` and `chain`. We'll learn more about `chain` later, for now, we're just interested in the parallels between mapping an `Option` and a `Promise`

In [ ]:
import * as P from './lib/promise'


const mappedPromise = F.pipe(
    P.of("I told you"),
    P.map(exclam)
)

console.log(await mappedPromise)

## Power ups, please!


We've seen the basics of two different data types, and how we can work with them.  
We'll learn more about different data types soon, but for now, the most important thing to know is that we can work with them using the same patterns, like `map`, and that each data type will have different semantics and use cases.  
Here are the most common data types you might come across and some of their use cases. Don't worry if some use cases sound confusing, just be aware of what they're used for  

* *Option*: optional values, nullability
* *Promise*: async behaviour
* *Array*: multiple values, non determinism
* *Either*: error handling
* *Reader*: dependency injection, "reading" from an environment
* *Writer*: logging
* *IO*: read/write from stdin/console, localstorage, etc

We will use `Either` and `Array` a lot more, so let's just add the types for those two `map`s as well
  

In [ ]:
import * as E from 'fp-ts/Either'

type MapArray = <A, B>(f: (a: A) => B) => (ma: Array<A>) => Array<B>
type EitherMap = <A,B>(f: (a: A) => B) => <E>(ma: E.Either<E, A>) => E.Either<E, B>